In [2]:
#dataframe
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.functions import col, avg, max
spark = SparkSession.builder.appName("DemoApp").getOrCreate()
data = [("John",24,"HR", 50000), ("Ria", 22, "DEv", 450000), ("Dave", 26,"Devops", 55000)]
#inferschema 
schema = StructType([StructField("name", StringType(), True), StructField("age", IntegerType(), True), 
                     StructField("Role", StringType(), True), StructField("salary", IntegerType(), True)])

df = spark.createDataFrame(data, schema)
#df_csv = spark.read.csv("./Documents/users.csv", header = True, inferSchema = True)
#df_csv.show()
#df_csv.select("name","age").filter(df_csv.age > 18).show()
#df_updated = df_csv.withColumn("retirement",60 - col("age"))
#df.withColumnRenamed("","")
#df.drop("location")
#df_updated.show()
#df_csv.groupBy("location").agg(avg("age").alias("Average_Age"), max("age").alias("Maximum Age")).show()
roles = [("max","developer"),("john","hr"),("ria","UI/UX"), ("max", "devops"), ("david", "QA")]
schema = StructType([StructField("name", StringType(), True), StructField("role", StringType(), True)])
df_roles = spark.createDataFrame(roles, schema)
#df_joined = df_csv.join(df_roles, "name", "inner")
#df_joined.show()
#df.dropna()
#df.fillna({"location":"Chennai"})
#df_csv.createOrReplaceTempView("people")
#res = spark.sql("SELECT name, age FROM people WHERE age > 18")
#res.orderBy("age", ascending = False).show()
#res.write.csv("./results.csv", mode = "overwrite")
df.show()
df.printSchema() 

+----+---+------+------+
|name|age|  Role|salary|
+----+---+------+------+
|John| 24|    HR| 50000|
| Ria| 22|   DEv|450000|
|Dave| 26|Devops| 55000|
+----+---+------+------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Role: string (nullable = true)
 |-- salary: integer (nullable = true)



In [1]:
"""Problem 1: Hogwarts House Points Analysis 
Problem Description:
At Hogwarts School of Witchcraft and Wizardry, the four houses (Gryffindor, Hufflepuff, Ravenclaw, Slytherin) 
earn points for their achievements throughout the year. You’ve been tasked by Professor Dumbledore to analyze 
the house points data to determine which house is leading and summarize points by house and year. 
Select the relevant columns (house, year, points), filter for houses with more than 50 points in a 
given year, group by house and year to sum the points, and order the results by year 
and total points in descending order to see who’s dominating the House Cup race. 
The Great Hall is buzzing with anticipation for these results."""

from pyspark.sql import SparkSession
from pyspark.sql.functions import sum
spark = SparkSession.builder.appName("HogwartsAnalysis").getOrCreate()

data = [
    ("Gryffindor", 1, 80, "Harry Potter"),
    ("Slytherin", 1, 60, "Draco Malfoy"),
    ("Ravenclaw", 1, 45, "Luna Lovegood"),
    ("Hufflepuff", 1, 30, "Cedric Diggory"),
    ("Gryffindor", 2, 90, "Hermione Granger"),
    ("Slytherin", 2, 70, "Pansy Parkinson"),
    ("Ravenclaw", 2, 55, "Cho Chang"),
    ("Hufflepuff", 2, 65, "Hannah Abbott"),
    ("Gryffindor", 3, 20, "Ron Weasley"),
    ("Slytherin", 3, 85, "Blaise Zabini")
]

columns = ["house", "year", "points", "student"]

df = spark.createDataFrame(data, columns)
res=(df.select("house","year","points").filter(df.points>50).groupBy("house","year").agg(sum("points").alias("total_points")).orderBy("year","total_points", ascending=False))
res.show() 


+----------+----+------------+
|     house|year|total_points|
+----------+----+------------+
| Slytherin|   3|          85|
|Gryffindor|   2|          90|
| Slytherin|   2|          70|
|Hufflepuff|   2|          65|
| Ravenclaw|   2|          55|
|Gryffindor|   1|          80|
| Slytherin|   1|          60|
+----------+----+------------+



In [2]:
"""Problem 2: Fellowship of the Ring Battle Stats 
Problem Description:
The Fellowship of the Ring is battling across Middle-earth, and Gandalf needs your help 
to analyze their combat performance. The data tracks each member’s battles, enemies defeated, and injuries sustained.
To prepare for the next council in Rivendell, select the member’s name, race, and enemies defeated, 
filter for battles where more than 5 enemies were defeated, group by race to calculate the average enemies defeated,
and order by the average in descending order. This will help the council decide which races are most effective in
combat and plan their strategy against Sauron’s forces."""

from pyspark.sql import SparkSession
from pyspark.sql.functions import avg
spark = SparkSession.builder.appName("RingBattleStats").getOrCreate()

data = [
    ("Aragorn", "Human", 10, 2, "Helms Deep"),
    ("Legolas", "Elf", 15, 0, "Helms Deep"),
    ("Gimli", "Dwarf", 8, 3, "Helms Deep"),
    ("Frodo", "Hobbit", 2, 1, "Moria"),
    ("Sam", "Hobbit", 4, 2, "Moria"),
    ("Gandalf", "Wizard", 12, 1, "Moria"),
    ("Boromir", "Human", 7, 4, "Amon Hen"),
    ("Legolas", "Elf", 20, 0, "Amon Hen"),
    ("Aragorn", "Human", 9, 2, "Amon Hen")
]

columns = ["name", "race", "enemies_defeated", "injuries", "battle"]
df = spark.createDataFrame(data, columns)
res1=(df.select("name","race","enemies_defeated").filter(df.enemies_defeated > 5).groupBy("race").agg(avg("enemies_defeated").alias("avg_enemies")).orderBy("avg_enemies", ascending=False))
res1.show()

+------+-----------------+
|  race|      avg_enemies|
+------+-----------------+
|   Elf|             17.5|
|Wizard|             12.0|
| Human|8.666666666666666|
| Dwarf|              8.0|
+------+-----------------+



In [3]:
"""Problem 3: Submarine Nuclear Warhead Deployment 
Problem Description:
You’re the data officer aboard the USS Nautilus, a nuclear submarine tasked with managing warhead deployments. 
The admiral needs a report to evaluate launch efficiency across missions.
Select the submarine ID, mission name, and warheads launched, filter for missions where more than 3 warheads
were launched successfully, group by submarine ID to count the total missions and sum the warheads launched,
and order by total warheads launched in descending order. This analysis will determine which submarines are ready 
for the next high-stakes operation in the Pacific."""

from pyspark.sql.functions import count, sum

data = [
    ("SUB-01", "Pacific Strike", 5, "Success"),
    ("SUB-02", "Atlantic Surge", 2, "Failure"),#
    ("SUB-01", "Arctic Blitz", 4, "Success"),
    ("SUB-03", "Indian Ocean", 6, "Success"),
    ("SUB-02", "Pacific Strike", 3, "Success"),#
    ("SUB-01", "Coral Sea", 7, "Success"),
    ("SUB-03", "Arctic Blitz", 1, "Failure"),#
    ("SUB-02", "Bering Strait", 5, "Success")
]

columns = ["submarine_id", "mission_name", "warheads_launched", "status"]
df = spark.createDataFrame(data, columns)
res = (df.select("submarine_id", "mission_name", "warheads_launched").filter((df.warheads_launched > 3) & (df.status == "Success"))
      .groupBy("submarine_id").agg(count("mission_name").alias("total_missions"), sum("warheads_launched").alias("total_warheads"))
    .orderBy("total_warheads", ascending = False))
res.show() 

+------------+--------------+--------------+
|submarine_id|total_missions|total_warheads|
+------------+--------------+--------------+
|      SUB-01|             3|            16|
|      SUB-03|             1|             6|
|      SUB-02|             1|             5|
+------------+--------------+--------------+

